In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


In [4]:
data_dir = 'C:/Users/SSC/OneDrive/Desktop/phase 4/Dataset'
train_dir = os.path.join(data_dir, 'Training')
test_dir = os.path.join(data_dir, 'Testing')
validation_dir = os.path.join(data_dir, 'Validation')

train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15


In [5]:
img_width, img_height = 224, 224
batch_size = 32


In [6]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)


Found 4200 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [7]:
# Load the pre-trained VGG16 model without the top classification layers
base_model = VGG16(weights='imagenet', include_top=False)

# Add custom layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


58889256/58889256 [==============================] - 74s 1us/step


In [8]:
epochs = 5
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping]
)


Epoch 1/5
132/132 [==============================] - 1197s 9s/step - loss: 0.6310 - accuracy: 0.6745 - val_loss: 0.6135 - val_accuracy: 0.6375
Epoch 2/5
132/132 [==============================] - 1150s 9s/step - loss: 0.5576 - accuracy: 0.7402 - val_loss: 0.5606 - val_accuracy: 0.6875
Epoch 3/5
132/132 [==============================] - 1029s 8s/step - loss: 0.5221 - accuracy: 0.7545 - val_loss: 0.5282 - val_accuracy: 0.7217
Epoch 4/5
132/132 [==============================] - 1276s 10s/step - loss: 0.4959 - accuracy: 0.7748 - val_loss: 0.5024 - val_accuracy: 0.7392
Epoch 5/5
132/132 [==============================] - 1919s 15s/step - loss: 0.4663 - accuracy: 0.7895 - val_loss: 0.5053 - val_accuracy: 0.7392


In [9]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_accuracy}")

# Make predictions on the test set
predictions = model.predict(test_generator)
predicted_classes = np.round(predictions)

# Calculate metrics and generate a classification report
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

print(f"Test Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


19/19 [==============================] - 155s 8s/step - loss: 0.5062 - accuracy: 0.7433
Test Accuracy: 0.7433333396911621
19/19 [==============================] - 192s 10s/step
Test Accuracy: 0.46
Precision: 0.44782608695652176
Recall: 0.3433333333333333
F1-Score: 0.38867924528301884
              precision    recall  f1-score   support

           F       0.47      0.58      0.52       300
           M       0.45      0.34      0.39       300

    accuracy                           0.46       600
   macro avg       0.46      0.46      0.45       600
weighted avg       0.46      0.46      0.45       600

